In [ ]:
!pip install d2l==1.0.0-beta0

Cho đến nay, có vẻ như chúng ta đã bỏ qua việc cẩu thả trong việc thiết lập mạng lưới của mình. Cụ thể, chúng ta đã thực hiện những điều không trực quan sau đây, có vẻ như chúng không nên hoạt động:
- Chúng ta đã xác định kiến ​​trúc mạng mà không chỉ định kích thước đầu vào.
- Chúng ta đã thêm các lớp mà không chỉ định kích thước đầu ra của lớp trước đó.
- Chúng tôi thậm chí còn “khởi tạo” các thông số này trước khi cung cấp đủ thông tin để xác định số lượng thông số mà mô hình của chúng tôi nên chứa.

Ta có thể ngạc nhiên rằng mã của chúng ta hoàn toàn chạy. Xét cho cùng, không có cách nào mà khung học sâu có thể cho biết kích thước đầu vào của mạng sẽ như thế nào. Mẹo ở đây là khung trì hoãn quá trình khởi tạo , đợi cho đến lần đầu tiên chúng tôi chuyển dữ liệu qua mô hình, để suy ra kích thước của từng lớp một cách nhanh chóng.

Sau này, khi làm việc với các mạng thần kinh tích chập, kỹ thuật này sẽ càng trở nên thuận tiện hơn vì kích thước đầu vào (nghĩa là độ phân giải của hình ảnh) sẽ ảnh hưởng đến kích thước của từng lớp tiếp theo. Do đó, khả năng đặt tham số mà không cần biết, tại thời điểm viết mã, kích thước là gì có thể đơn giản hóa rất nhiều nhiệm vụ chỉ định và sau đó sửa đổi các mô hình của chúng ta. Tiếp theo, chúng ta đi sâu hơn vào cơ chế khởi tạo.

In [2]:
import torch
from torch import nn
from d2l import torch as d2l

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Để bắt đầu, hãy khởi tạo một MLP.

In [3]:
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Tại thời điểm này, mạng không thể biết kích thước trọng số của lớp đầu vào vì kích thước đầu vào vẫn chưa được biết.

Do đó, khung chưa khởi tạo bất kỳ tham số nào. Chúng ta xác nhận bằng cách thử truy cập các thông số bên dưới.

In [4]:
net[0].weight

<UninitializedParameter>

Tiếp theo, hãy truyền dữ liệu qua mạng để làm cho khuôn khổ cuối cùng khởi tạo các tham số.

In [5]:
X = torch.rand(2, 20)
net(X)

net[0].weight.shape

torch.Size([256, 20])

Ngay khi chúng ta biết kích thước đầu vào, 20, khung có thể xác định hình dạng của ma trận trọng số của lớp đầu tiên bằng cách nhập giá trị 20. Sau khi nhận ra hình dạng của lớp đầu tiên, khung sẽ chuyển sang lớp thứ hai, v.v. biểu đồ tính toán cho đến khi tất cả các hình dạng được biết đến. Lưu ý rằng trong trường hợp này, chỉ lớp đầu tiên yêu cầu khởi tạo chậm, nhưng khung khởi tạo tuần tự. Khi tất cả các hình dạng tham số đã được biết, khung cuối cùng có thể khởi tạo các tham số.

Phương pháp sau chuyển các đầu vào giả qua mạng để chạy khô để suy ra tất cả các hình dạng tham số và sau đó khởi tạo các tham số. Nó sẽ được sử dụng sau này khi không muốn khởi tạo ngẫu nhiên mặc định.

In [7]:
@d2l.add_to_class(d2l.Module)
def apply_init(self, inputs, init=None):
    self.forward(*inputs)
    if init is not None:
        self.net.apply(init)